# Topic Modelling for News

![](https://images.unsplash.com/photo-1495020689067-958852a7765e?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=1050&q=80)

Photo by [Roman Kraft](https://unsplash.com/photos/_Zua2hyvTBk)

This exercise is about modelling the main topics of a database of News headlines.

Begin by importing the needed libraries:

In [2]:
# TODO: import needed libraries
import nltk
import pandas as pd
import numpy as np


Load the data in the file `random_headlines.csv`

In [3]:
# TODO: load the dataset
df = pd.read_csv('random_headlines.csv')
print(df.shape)
df.head()

(20000, 2)


,publish_date,headline_text
0,20120305,ute driver hurt in intersection crash
1,20081128,6yo dies in cycling accident
2,20090325,bumper olive harvest expected
3,20100201,replica replaces northernmost sign
4,20080225,woods targets perfect season


This is always a good idea to perform some EDA (exploratory data analytics) on a dataset...

In [4]:
# TODO: Perform a short EDA
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   publish_date   20000 non-null  int64 
 1   headline_text  20000 non-null  object
dtypes: int64(1), object(1)
memory usage: 312.6+ KB


Now perform all the needed preprocessing on those headlines: case lowering, tokenization, punctuation removal, stopwords removal, stemming/lemmatization.

In [9]:
# TODO: Preprocess the input data

import nltk
nltk.download()


# tokenize
df['tokens'] = df['headline_text'].apply(lambda row: nltk.word_tokenize(row))

# punctuation
df['alphanumeric'] = df['tokens'].apply(lambda row: [
    word for word in row if word.isalpha()
])

# remove stopwords
stop = nltk.corpus.stopwords.words('english')
df['stop'] = df['alphanumeric'].apply(lambda row: [
    word for word in row if word not in stop
])

# stemming
stemmer = nltk.PorterStemmer()
df['stemmed'] = df['stop'].apply(lambda row: [
    stemmer.stem(word) for word in row
])

df['stemmed'].head()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


0    [ute, driver, hurt, intersect, crash]
1                       [die, cycl, accid]
2          [bumper, oliv, harvest, expect]
3    [replica, replac, northernmost, sign]
4          [wood, target, perfect, season]
Name: stemmed, dtype: object

Now use Gensim to compute a BOW

In [14]:
# TODO: Compute the BOW using Gensim
from gensim.corpora import Dictionary

dictionary = Dictionary(df['stemmed'])
corpus = [dictionary.doc2bow(line) for line in df['stemmed']]
##print(np.shape(corpus))
##corpus[0:2]
print(len(corpus))
print([len(doc) for doc in corpus[0:2]])


20000
[5, 3]


Compute the TF-IDF using Gensim

In [17]:
# TODO: Compute TF-IDF
from gensim.models import TfidfModel

tfidf_model = TfidfModel(corpus)
tf_idf = tfidf_model[corpus]
##print(np.shape(tf_idf))
print(len(tf_idf))
print([len(doc) for doc in tf_idf[:5]])


20000
[5, 3, 4, 4, 4]


Finally compute the **LSA** (also called LSI) using Gensim, for a given number of Topics that you choose yourself

In [18]:
# TODO: Compute LSA
from gensim.models import LsiModel

lsi = LsiModel(corpus=corpus, num_topics=4, id2word=dictionary)


For each of the topic, show the most significant words.

In [19]:
# TODO: Print the 3 or 4 most significant words of each topic
lsi.print_topics(num_words=3)

[(0, '-0.752*"polic" + -0.404*"man" + -0.207*"charg"'),
 (1, '-0.670*"man" + 0.574*"polic" + -0.329*"charg"'),
 (2, '-0.656*"new" + -0.296*"plan" + 0.240*"man"'),
 (3, '-0.702*"new" + 0.345*"say" + 0.333*"plan"')]

What do you think about those results?

Now let's try to use LDA instead of LSA using Gensim

In [20]:
# TODO: Compute LDA
from gensim.models import LdaModel

lda = LdaModel(corpus=corpus, num_topics=4, id2word=dictionary)

In [21]:
# TODO: print the most frequent words of each topic
lda.print_topics(num_words=3)

[(0, '0.016*"man" + 0.015*"polic" + 0.012*"charg"'),
 (1, '0.008*"say" + 0.007*"back" + 0.007*"interview"'),
 (2, '0.007*"new" + 0.007*"polic" + 0.005*"second"'),
 (3, '0.006*"govt" + 0.006*"year" + 0.006*"mine"')]

Now, how does it work with LDA?

Let's make some visualization of the LDA results using pyLDAvis.

In [28]:
!pip install pyLDAvis


In [30]:
# TODO: show visualization results of the LDA
import pyLDAvis
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

vis = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
vis


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.112469 -0.084007       1        1  26.559086
0     -0.226842 -0.056550       2        1  25.950972
3      0.093489 -0.086981       3        1  23.963540
2      0.020884  0.227538       4        1  23.526402, topic_info=       Term        Freq       Total Category  logprob  loglift
139     man  463.000000  463.000000  Default  30.0000  30.0000
86    charg  338.000000  338.000000  Default  29.0000  29.0000
185   polic  629.000000  629.000000  Default  28.0000  28.0000
312   court  287.000000  287.000000  Default  27.0000  27.0000
640    back  231.000000  231.000000  Default  26.0000  26.0000
...     ...         ...         ...      ...      ...      ...
1858     pm   61.339257  107.412937   Topic4  -5.9751   0.8868
368      wa   64.831223  188.394970   Topic4  -5.9197   0.3803
54      nsw   63.075228  185.097305   Topic4  -5.9471   0.3705
283   boost   61.192684  141.947437   Topic4  -5.9775   0.6056
459    home   60.836929  162.982549   Topic4  -5.9833   0.4616

[306 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
1103      1  0.993661   abbott
496       1  0.018149      abc
496       2  0.970955      abc
496       3  0.009074      abc
496       4  0.009074      abc
...     ...       ...      ...
2641      1  0.982911    worri
251       1  0.006578     year
251       2  0.059200     year
251       3  0.934046     year
3465      4  0.988436  zealand

[604 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 4, 3])

Depending on your results, you can try to fine tune the algorithm: number of topics, hyperparameters...
And check with others their results.